In [ ]:
import os
os.environ['OMP_NUM_THREADS'] = '30'

from qutip import *
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from scipy.optimize import curve_fit


def func_polynomial(x, a, b, c):
    return a * x**(b) + c

def func_eig(x, a, b, c):
    return a * x**(-b/2) + c

In [ ]:
def parallel_ground_state(N):

    J_x            = jmat( N/2, 'x')
    J_z            = jmat( N/2, 'z')
    J_minus        = jmat( N/2, '-')

    kappa          = 1
    c_ops          = [np.sqrt( kappa / (N/2) ) * J_minus]

    #From Figure 2 - BTC file:
    QFI_vec       = np.load(folder_name + "QFI_along_w0_N="+str(N)+".npy")
    w0_vec        = np.load(folder_name + "w0_N="+str(N)+".npy")

    w0_at          = w0_vec[np.argmax(QFI_vec)]

    #Second eigenvalue of the Liouvillian:    
    H_at           = w0_at * J_x
    L_at           = liouvillian(H_at, c_ops)
    eig_mat_at     = L_at.eigenenergies(sparse = True, sort='high', eigvals=2, tol=0, maxiter=100000)
    eig_2nd_at     = np.abs(np.real(eig_mat_at)[1])

    at_time        = alpha_coefficient * (1 / eig_2nd_at)

    rho0           = ket2dm( (J_z.groundstate()[1]).unit() )
                            
    tlist          = np.linspace(0, at_time, 400)

    div            = 1000
    
    delta_w0_at    = w0_at / div

    ##
    rhot_p         = mesolve( (w0_at + delta_w0_at) * J_x, rho0, tlist, c_ops, [])
    rhot_p         = rhot_p.states[-1]

    ##
    rhot_m         = mesolve( (w0_at - delta_w0_at) * J_x, rho0, tlist, c_ops, [])
    rhot_m         = rhot_m.states[-1]

    QFI            = 8 * ( 1 - fidelity(rhot_p, rhot_m) ) / (2 * delta_w0_at)**2

    np.save(folder_name_save + "eig_at_N="+str(N)+"_alpha_coeff="+str(alpha_coefficient)\
            , eig_2nd_at )
    np.save(folder_name_save + "qfi_at_N="+str(N)+"_alpha_coeff="+str(alpha_coefficient)\
            , QFI )

In [ ]:
N_vec_1                 = range(2  , 100, 2)
N_vec_2                 = range(100, 300, 20)

N_vec                   = np.concatenate( (N_vec_1, N_vec_2), axis = 0 )

alpha_coefficient_vec   = [1, 2, 5]

for alpha_coefficient in alpha_coefficient_vec:
    
    print("alpha = ", alpha_coefficient)
    
    parfor(parallel_ground_state, N_vec)